In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [61]:
def generate_tgl(tgl_awal, tgl_akhir):
    tgl_ambil=[]
    for i in range(tgl_awal,tgl_akhir+1):
        if i<10:
            tgl_ambil.append('0'+str(i))
        else:
            tgl_ambil.append(str(i))
    return tgl_ambil

In [62]:
def scraping_compas(key, tgl_awal, tgl_akhir):
    tgl_ambil=generate_tgl(tgl_awal, tgl_akhir)
    key_tgl=['https://indeks.kompas.com/{}/2019-05-{}'.format(key, j) for j in tgl_ambil]
    hasil=[]
    for j in key_tgl:
        req=requests.get(j)
        soup=BeautifulSoup(req.text, 'lxml')
        kumpulan_page=soup.find_all('div',class_='article__asset')
        link_page=[kumpulan_page[i].a['href'] for i in range(len(kumpulan_page))]
        for i in link_page:
            item={}
            req_berita=requests.get(i)
            soup_berita=BeautifulSoup(req_berita.text, 'lxml')
            item['judul']=soup_berita.find_all('title')[0].text
            tgl=soup_berita.find_all('div', {'class':'read__time'})[0].text
            item['tanggal']=tgl.split(',')[0].split('-')[1].replace(' ',"")
            berita=soup_berita.find_all('div', {'class':'read__content'})[0]
            item['paragrap']=berita.text.strip()
            photo=soup_berita.find_all('div', {'class':'photo'})[0]
            item['photo']=photo.img['src']
            try:
                item['Youtube']=berita.find('iframe')['src']
            except: 
                item['Youtube']=None
            hasil.append(item)
    return hasil

In [63]:
key='tekno'
tgl_awal=6
tgl_akhir=10
df=pd.DataFrame(scraping_compas(key, tgl_awal, tgl_akhir))

In [64]:
df.head()

,Youtube,judul,paragrap,photo,tanggal
0,None,Menanti Android Q dan Pixel 3a di Google I/O 2019,KOMPAS.com - Google I/O merupakan sebuah ajang...,https://asset.kompas.com/crop/81x0:666x390/750...,06/05/2019
1,None,"Dari Samsung hingga Oppo, Ini Deretan Ponsel 5...",KOMPAS.com - Teknologi jaringan seluler gener...,https://asset.kompas.com/crop/0x0:940x627/750x...,06/05/2019
2,https://www.youtube.com/embed/bKNFooao3GA,"Vivo S1 Pro Resmi Meluncur, Bedanya dengan S1 ...",KOMPAS.com - Vivo kembali merilis seri baru d...,https://asset.kompas.com/crop/21x15:729x487/75...,06/05/2019
3,None,Indosat Ooredoo Tunjuk Ahmad Abdulaziz Al Neam...,KOMPAS.com - Indosat Ooredoo resmi menunjuk A...,https://asset.kompas.com/crop/0x83:1000x750/75...,06/05/2019
4,None,DJI Siapkan Kamera Aksi Pesaing Go Pro?,KOMPAS.com - DJI dikenal sebagai pabrikan dro...,https://asset.kompas.com/crop/65x0:882x545/750...,06/05/2019


In [65]:
df.to_csv('berita_kompas.csv')